In [22]:
#Import spark related libraries
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [23]:
spark = SparkSession.builder.appName("Demo").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("s3a://aws-bda-assign1/crimes_sample.csv")

In [24]:
data = df[df['Year'] == 2007].groupby(df['Primary Type']).count()

In [25]:
data.show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|OFFENSE INVOLVING...|   42|
|            STALKING|    7|
|PUBLIC PEACE VIOL...|   34|
|               ARSON|    4|
|            GAMBLING|    7|
|   CRIMINAL TRESPASS|  119|
|             ASSAULT|  225|
|LIQUOR LAW VIOLATION|   11|
| MOTOR VEHICLE THEFT|  186|
|               THEFT|  918|
|             BATTERY|  793|
|             ROBBERY|  142|
| CRIM SEXUAL ASSAULT|   14|
|        PROSTITUTION|   94|
|  DECEPTIVE PRACTICE|  215|
|         SEX OFFENSE|    9|
|     CRIMINAL DAMAGE|  522|
|           NARCOTICS|  732|
|       OTHER OFFENSE|  282|
|          KIDNAPPING|    1|
+--------------------+-----+
only showing top 20 rows



In [33]:
## Mysql Connector
import mysql.connector
import logging


def get_connection():
    """
    get MySQL connection.  auto-commit is set to False by default.  you don't need to
    start a transaction explicitly
    :param environment: the environment
    :param read_only: whether to connect to read-only db or not
    :param db: name of the database
    :return: the database connection
    """
    config = {
        'user': 'asad',
        'password': 'admin123',
        'host': 'myawsbigdatainstance.cmioopaskdwr.us-east-1.rds.amazonaws.com',
        'database': 'crimes_database'
    }

    cn = mysql.connector.connect(**config)
    return cn


def execute_query_with_cnx(cnx, sql, parameters=(), fetch_one=False, close_connection=True):
    """
    execute SQL query with given connection
    :param close_connection: if True close the connection after the query is ran
    :param cnx: database connection you opened explicitly
    :param sql: SQL query to execute
    :param parameters: parameters - defaults to empty
    :param fetch_one: fetch only single record
    :return:
    """
    # return dictionary rather than tuple
    cursor = cnx.cursor(dictionary=True)
    cursor.execute(sql, parameters)
    everything = cursor.fetchone() if fetch_one else cursor.fetchall()
    cursor.close()

    if close_connection:
        cnx.close()
    return everything


def execute_update_with_cnx(cnx, sql, parameters=()):
    """
    execute SQL update with given connection.
    Wasted a lot of time in it so adding this as a reminder, please commit your changes manually.
    :param cnx: DB connection
    :param sql: SQL query to execute
    :param parameters: parameters - defaults to empty
    :return:
    """
    cursor = cnx.cursor()
    cursor.execute(sql, parameters)
    cursor.close()


In [34]:
conn = get_connection()

In [35]:
for crimes_data in data.collect():
    print(crimes_data[0], crimes_data[1])
    insert_query = """INSERT INTO crimes (crime, count) VALUES('%s', '%i'); """ % (crimes_data[0], crimes_data[1])
    execute_update_with_cnx(conn, insert_query)
    conn.commit()

(u'OFFENSE INVOLVING CHILDREN', 42)
(u'STALKING', 7)
(u'PUBLIC PEACE VIOLATION', 34)
(u'ARSON', 4)
(u'GAMBLING', 7)
(u'CRIMINAL TRESPASS', 119)
(u'ASSAULT', 225)
(u'LIQUOR LAW VIOLATION', 11)
(u'MOTOR VEHICLE THEFT', 186)
(u'THEFT', 918)
(u'BATTERY', 793)
(u'ROBBERY', 142)
(u'CRIM SEXUAL ASSAULT', 14)
(u'PROSTITUTION', 94)
(u'DECEPTIVE PRACTICE', 215)
(u'SEX OFFENSE', 9)
(u'CRIMINAL DAMAGE', 522)
(u'NARCOTICS', 732)
(u'OTHER OFFENSE', 282)
(u'KIDNAPPING', 1)
(u'BURGLARY', 230)
(u'WEAPONS VIOLATION', 42)
(u'INTERFERENCE WITH PUBLIC OFFICER', 8)
